# Explore IBL data Pipeline

Here we introduce some useful DataJoint tools to explore the IBL data pipeline

First thing first, let's **import DataJoint and the IBL pipeline package**.

In [ ]:
import datajoint as dj
from ibl_pipeline import reference, subject, action, acquisition, data, behavior, ephys, histology
from ibl_pipeline.analyses import behavior as behavior_analyses

# 1. Browse schemas you have access to - `dj.list_schemas()`

In [ ]:
dj.list_schemas()

## Schema types

`ibl_*`: common schemas that regular users just have the read access to it.
`group_shared_*`: schemas created by users, every user has read and write access to it. This is a place for people to share their research with other people in the collaboration.
`user_{your_user_name}_*`: schemas created by a particular user. Only that user and admin has read and write access to that schema. This is your private space for your own analyses. If you would like to share a particular schema to other people, please contact the admin (Shan) to grant users privilege to access that schema.

## Major schemas: 

`ibl_reference`: lab, user, project and the general information such as brain regions.
`ibl_subject`: colony management system of subjects.
`ibl_action`: mainly the watering information of the animals.
`ibl_acquisition`: session information.
`ibl_data`: data repository, dataset and file uploading records.
`ibl_behavior`: behavior trial information.
`ibl_analyses_behavior`: pre-computed behavioral summary.
`ibl_ephys`: ephys data, e.g. clusters.
`ibl_histology`: histological data, probe trajectory and brain region assignment on each cluster.
`ibl_qc`: quality check results on session level and probe level.

**Note**: each module we imported from `ibl_pipeline` corresponds to a schema in the database. This is because there is a schema object in each of these modules. For example:

In [ ]:
subject.schema

This object talks to the schema `ibl_subject` in the database.

Inside the module, there are many classes that corresponds to individual tables in this schema, for example:

In [ ]:
subject.Subject()

Underlying, it connects to the table in the schema inside the database.

In [ ]:
subject.Subject.full_table_name

# 2. Browse tables in a schema - `dj.Diagram`

**Table tiers**:  
Manual table: green box  
Lookup table: gray box  
Imported table: blue oval  
Computed table: red circle  
Part table: plain text

**Dependencies**:  
One-to-one primary: thick solid line  
One-to-many primary: thin solid line  
Secondary foreign key reference: dashed line
Renamed secondary foreign key references: orange dot

## Show tables in the whole schema

In [ ]:
dj.Diagram(reference)

## Show diagram of arbitruary parts of the database

In [ ]:
dj.Diagram(subject)

A combination of arbitruary tables:

In [ ]:
dj.Diagram(subject.Subject) + dj.Diagram(subject.Line) + dj.Diagram(subject.SubjectLab) + dj.Diagram(subject.SubjectProject)

The tables could be from different schemas:

In [ ]:
dj.Diagram(subject.Subject) + dj.Diagram(acquisition.Session)

# 3. Getting the detailed definition of a table - `table.describe()`

In [ ]:
subject.Subject.describe();

# 4. Browsing of data - queries

Query all subjects

In [ ]:
subject.Subject()

## Restriction  `&`: filtering data

### Restriction: Query one subject

In [ ]:
# restrict by string
subject.Subject & 'subject_nickname="ibl_witten_10"'

In [ ]:
# restrict by dictionary
from uuid import UUID
subject.Subject & {'subject_uuid': UUID('00c60db3-74c3-4ee2-9df9-2c84acf84e92')}

In [ ]:
subject.Subject & {'sex': 'm'}

### Restriction: Query subjects born after a date

In [ ]:
subject.Subject & 'subject_birth_date > "2019-01-01"'

### Restriction: subjects within a range of dates

In [ ]:
subject.Subject & 'subject_birth_date between "2019-01-01" and "2019-04-01"'

### Restriction: Query subjects on multiple attributes

In [ ]:
subject.Subject & 'subject_birth_date > "2019-01-01"' & 'sex="M"'

### Restriction: Query subjects restricted by other tables

In [ ]:
# subjects with Ephys recording
subject.Subject & ephys.ProbeInsertion

In [ ]:
# subjects without Ephys recording
subject.Subject - ephys.ProbeInsertion

## Join `*`:  gather information from different tables

In [ ]:
subject.Subject * acquisition.Session

## Projection `.proj()`: focus on attributes of interest

In [ ]:
subject.Subject.proj()

In [ ]:
subject.Subject.proj('subject_birth_date', 'sex')

### rename attribute with ***proj()***

In [ ]:
subject.Subject.proj('sex', dob='subject_birth_date')

### perform simple computations with ***proj***

**Example 1: Get date of a session:**

In [ ]:
sessions_with_date = acquisition.Session.proj(session_date='date(session_start_time)')

In [ ]:
sessions_with_date

**Example 2: Age of the animal when performing each session?**

In [ ]:
# First get the date of birth and the session date into the same query
q = subject.Subject * acquisition.Session

In [ ]:
q

In [ ]:
# Then compute the age
q_with_age = q.proj(age='datediff(session_start_time, subject_birth_date)')
q_with_age

## Aggregation `.aggr()`: simple computation of one table against another table

Example: how many sessions does each subject do so far?

In [ ]:
subject.Subject.aggr(acquisition.Session, 'subject_nickname', n='count(*)')

# 5. Fetching data

## Fetch all fields: `fetch()`

In [ ]:
# fetch all data from a table
subjs = subject.Subject.fetch()
subjs

In [ ]:
subjs['subject_uuid']

In [ ]:
subjs['subject_birth_date']

In [ ]:
# fetch as a list of dictionaries
subjs_dict = subject.Subject.fetch(as_dict=True)
subjs_dict

In [ ]:
# fetch as pandas dataframe
subjs_df = subject.Subject.fetch(format='frame')
subjs_df

In [ ]:
# fetch the primary key
pk = subject.Subject.fetch('KEY')
pk

In [ ]:
# fetch specific attributes
dob, sex = subject.Subject.fetch('subject_birth_date', 'sex')

In [ ]:
dob

In [ ]:
info = subject.Subject.fetch('subject_birth_date', 'sex', as_dict=True)
info

## fetch data only from one entry: `fetch1`

In [ ]:
ibl_witten_10 = (subject.Subject & {'subject_nickname': 'ibl_witten_10'}).fetch1('KEY')  # "fetch1()" because we know there's only one

In [ ]:
ibl_witten_10

In [ ]:
IBL_10 = (subject.Subject & {'subject_nickname': 'IBL_10'}).fetch1()

In [ ]:
IBL_10

# 6. How to work with data where you don't have the code to generate the class?

In [ ]:
import datajoint as dj
dj.list_schemas()

Using `dj.create_virtual_module`, you'll be able to reconstruct the virtual module to access the tables in a schema.

In [ ]:
# the first argument is the `__name__` of this module, the second argument is the schema name.
anne_analyses = dj.create_virtual_module('analyses', 'group_shared_anneurai_analyses')

Now you could check the Diagram of this module

In [ ]:
dj.Diagram(anne_analyses)

Or do any query you want:

In [ ]:
anne_analyses.Age()

Another choice is to use `spawn_missing_classes`

In [ ]:
schema = dj.schema()